In [ ]:
import pandas as pd
import geopandas as gpd

# read the shape file
sf = gpd.read_file("../data/raw/PTV_METRO_BUS_STOP.shp")

sf.head()

In [ ]:
# Convert the geometry shaape to to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()

In [ ]:
# create a JSON 
geoJSON = sf['geometry'].to_json()

# print the first 300 chars of the json
print(geoJSON[:300])

In [ ]:
import folium

# plot the aggregated data
_map = folium.Map(location=[143.73, -37.56], tiles="Stamen Terrain", zoom_start=10)

_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))

_map


# save the plot
_map.save('../plots/bus_station.html')

In [ ]:
# select the columns needed
bus_station = sf[['STOP_NAME', 'LATITUDE', 'LONGITUDE']]

bus_station.head()

In [ ]:
from pandas.core.frame import DataFrame

# Create new list for later use
bus_station_list = []

# Loop through the dataframe, add unformation into the list
for i in range(0,len(bus_station)):
    row = bus_station.iloc[i]
    location = str(row[1]) + ', ' + str(row[2])
    bus_station_list.append([row[0], location])

# Forming new dataframe
bus_station_new = DataFrame(data = bus_station_list, columns=['Bus_Station_Name', 'Location'])

bus_station_new.head()


In [ ]:
pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

# create a list to store the bus station name, location and postcode of each bus station
postcode_list = []
geolocator = Nominatim(user_agent="vscode")

# count the number bus stations without postcode
count = 0

# extract the postcode of each bus station
for i in range(0,len(bus_station_new)):
    row = bus_station_new.iloc[i]
    location = geolocator.reverse(row[1])

    # check whether the postcode in the address
    if 'postcode' in location.raw['address'].keys():

        # get the postcode of the bus station
        post = location.raw['address']['postcode']

        #store the bus station name, location and postcode of the bus station in the list
        postcode_list.append([row[0], row[1], int(post)])
        
    else:
        count += 1

# convert the list to a dataframe
bus_station_postcode = DataFrame(data = postcode_list, columns=['Bus_Station_Name', 'Location', 'Postcode'])

bus_station_postcode.head()

In [ ]:
# save the dataframe
bus_station_postcode.to_csv('../data/curated/bus_station_postcode.csv')